In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install spicy
from scipy import stats

In [3]:
df = pd.read_csv('/home/isatish/projects/ab_testing_simulation/data/ab_merged_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted,country
0,851104,11:48.6,control,old_page,0,US
1,804228,01:45.2,control,old_page,0,US
2,661590,55:06.2,treatment,new_page,0,US
3,853541,28:03.1,treatment,new_page,0,US
4,864975,52:26.2,control,old_page,1,US


In [6]:
# compute conversion rates
summary = df.groupby("group")["converted"].agg(["count", "sum"])
summary["conversion_rate"] = summary["sum"] / summary["count"]
print(summary)

            count    sum  conversion_rate
group                                    
control    147202  17723         0.120399
treatment  147278  17514         0.118918


In [7]:
# extract group-level counts
control_success = summary.loc["control", "sum"]
control_total = summary.loc["control", "count"]
treat_success = summary.loc["treatment", "sum"]
treat_total = summary.loc["treatment", "count"]

In [15]:
# # z-test for proportions two tailed test
# def z_test_proportions(success_a, total_a, success_b, total_b):
#     p1 = success_a / total_a
#     p2 = success_b / total_b
#     p_pool = (success_a + success_b) / (total_a + total_b)
#     se = np.sqrt(p_pool * (1 - p_pool) * (1/total_a + 1/total_b))
#     z = (p1 - p2) / se
#     p_value = 2 * (1 - stats.norm.cdf(abs(z)))
#     alpha = 0.05
#     if p_value < alpha:
#         print("✅ Reject H0 → Treatment conversion is significantly higher")
#     else:
#         print("❌ Fail to reject H0 → No significant difference")
#     return z, p_value  

# z_test_proportions(control_success, control_total, treat_success, treat_total)

In [16]:
p1 = treat_success / treat_total
p2 = control_success / control_total
p_pool = (treat_success + control_success) / (treat_total + control_total)
se = np.sqrt(p_pool * (1 - p_pool) * (1 / treat_total + 1 / control_total))
z_score = (p1 - p2) / se
p_value = 1 - stats.norm.cdf(z_score)  # one-tailed test
print(f"Z-score: {z_score}, P-value: {p_value}")

Z-score: -1.2382796210706355, P-value: 0.8921938012768196


In [17]:
alpha = 0.05
if p_value < alpha:
    print("✅ Reject H0 → Treatment conversion is significantly higher")
else:
    print("❌ Fail to reject H0 → No significant difference")

❌ Fail to reject H0 → No significant difference
